In [ ]:
!mkdir dataset
!pip install gdown
!gdown 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
!mv refcocog.tar.gz ./dataset/
!ls dataset

mkdir: cannot create directory ‘dataset’: File exists
Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq 

mv: cannot stat 'refcocog.tar.gz': No such file or directory
refcocog  refcocog.tar.gz


In [ ]:
!tar -xf dataset/refcocog.tar.gz -C dataset
!ls dataset

refcocog  refcocog.tar.gz


In [ ]:
!pip install torch
!pip install torchvision
!pip install SummaryWriter
!pip install tensorboard
!pip install scikit-image
!pip install matplotlib
!pip install Lambda
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import os
from posixpath import split
import json
import tarfile
import io
import pickle
import sys
from PIL import Image
from collections import OrderedDict
from pathlib import Path
from collections import defaultdict


import numpy as np
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.utils import draw_bounding_boxes
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

device = "cuda" if torch.cuda.is_available() else "cpu"

ERROR: Could not find a version that satisfies the requirement SummaryWriter (from versions: none)
ERROR: No matching distribution found for SummaryWriter


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-mypq_nqz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-mypq_nqz
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done


In [ ]:
from clip import clip
'''torch.cuda._initialized=True
torch.cuda.is_available()

#return True

CUDA_VISIBLE_DEVICES=1
#i have only one gpu card

device = torch.cuda.current_device()
#return 0

#now comes to the error'''

model, preprocess = clip.load("RN50")
model = model.to(device).eval()

In [ ]:
class RefCOCOgDataset(Dataset):
    def __init__(self, dataset, transform, target_transform, data_dir='dataset/refcocog'):
        super(RefCOCOgDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform
        self.dataset = dataset


    def __getitem__(self, idx):
        fname = os.path.join(self.data_dir+'/images/', list(self.dataset.items())[idx][0])
        image = Image.open(fname)
        texts = list(self.dataset.items())[idx][1]
        data_item = {'image' : image, 'labels': texts}
        if self.transform:
            data_item['image'] = self.transform(image)
        if self.target_transform:
            data_item['labels'] = clip.tokenize(["This is " + sent for desc in texts for sent in desc])
        return data_item


    def __len__(self):
        return len(self.dataset)



In [ ]:
def get_datasets(refs, train = True):
  ref_data = []
  for val in refs:
    if train:
      if val['split'] == 'train' or val['split'] == 'val':
        ref_data.append(val)
    else:
      if val['split'] == 'test':
        ref_data.append(val)
  
  '''for i,v in enumerate(datasets):
    training_data, test_data = v'''

  return ref_data


def preProcess_datasets(data_dir, train):
  dataset = defaultdict(list)
  data = {}
  refs = {}     
  f = open(f'{data_dir}/annotations/refs(umd).p', 'rb')
  data['refs'] = pickle.load(f)
  if train:
    refs = get_datasets(data['refs'], train=True)
  else:
    refs = get_datasets(data['refs'], train=False)
  instances_file = os.path.join(f'{data_dir}/annotations/instances.json')
  instances = json.load(open(instances_file, 'r'))
  data['images'] = instances['images']
  data['annotations'] = instances['annotations'] 
  #print(data['refs'][0])
  #print(data['images'][0])
  #print(data['annotations'][0])
  for key,val in enumerate(data['images']):
    for v in refs:
      if(v['image_id'] == val['id']): 
          #for w in data['annotations']:
            #if(v['image_id'] == w['image_id']):
        fname = os.path.join(data_dir+'/images/', val['file_name'])
        if os.path.exists(fname):
          for inner_i, inner_v in enumerate(v['sentences']):
            dataset[val['file_name']].append(inner_v['sent'])
            
      else: 
        continue

  return dataset


  def visualise_result(img, txt):
    resize_image = T.Resize(100)
    img = resize_image(img)
    convert_tensor = T.ToTensor()
    image = convert_tensor(img).to(device)
    image = image.clone().detach()
    image = image.type(torch.ByteTensor).to(device)
    boxs = torch.tensor(w['bbox'],dtype=torch.int).to(device) 
    boxes = boxs.reshape([1,4])   
    boxes = torchvision.ops.box_convert(boxes, "cxcywh", "xyxy").to(device)
    image = draw_bounding_boxes(image=image, boxes=boxes, width=2, colors=(0,0,255), fill=True).to(device)
    image = image.permute(1,2,0)

In [ ]:
def encode_data(images_fp: list[str], texts: list[str], data_dir):
    # preprocess the images to transform from filenames to images to tensors
    images = [preprocess(Image.open(os.path.join(data_dir+'/images/',image))) for image in images_fp]

    # preprocess the texts to transform from text to tensors
    images = torch.tensor(np.stack(images)).to(device)

    text_tokens = clip.tokenize(["This is " + sent for desc in texts for sent in desc]).to(device)

    # encode the inputs
    with torch.no_grad():
        images_z = model.encode_image(images).float()
        texts_z = model.encode_text(text_tokens).float()
        texts_z /= texts_z.norm(dim=-1, keepdim=True)

    return images_z, texts_z

In [ ]:
def cosine_similarity(images_z: torch.Tensor, texts_z: torch.Tensor):
  # normalise the image and the text
  images_z /= images_z.norm(dim=-1, keepdim=True)
  texts_z /= texts_z.norm(dim=-1, keepdim=True)

  # evaluate the cosine similarity between the sets of features
  similarity = (texts_z @ images_z.T)

  return similarity.to(device)

In [ ]:
def training_step(net, data_loader, optimizer, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # set the network to training mode
    net.train()

  # iterate over the training set
    for batch_idx, batch in enumerate(data_loader):
        inputs = batch['image']
        targets = batch['labels']
        # load data into GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # forward pass
        outputs = net(inputs)

        # loss computation
        loss = cost_function(outputs, targets)

        # backward pass
        loss.backward()

        # parameters update
        optimizer.step()

        # gradients reset
        optimizer.zero_grad()

        # fetch prediction and loss value
        samples += inputs.shape[0]
        cumulative_loss += loss.item()
        _, predicted = outputs.max(dim=1) # max() returns (maximum_value, index_of_maximum_value)

        # compute training accuracy
        cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

def test_step(net, data_loader, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # set the network to evaluation mode
    net.eval() 

    # disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # iterate over the test set
        for batch_idx, batch in enumerate(data_loader):
            inputs = batch['image']
            targets = batch['labels']
            # load data into GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

            # forward pass
            outputs = net(inputs)

            # loss computation
            loss = cost_function(outputs, targets)

            # fetch prediction and loss value
            samples += inputs.shape[0]
            cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
            _, predicted = outputs.max(1)

            # compute accuracy
            cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

In [ ]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [ ]:
def get_optimizer(model, lr, wd, momentum):
  optimizer = torch.optim.SGD([
      {'params': model.classifier.parameters(), 'lr': lr}
  ], lr=lr / 10, weight_decay=wd, momentum=momentum)
  
  return optimizer

In [ ]:
"""
Applies Batch Normalization over a 1D input (or 2D tensor)

Shape:
  Input: (N, C)
  Output: (N, C)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm1d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm1d(4, affine=False)
  >>> input = torch.rand(10, 4)
  >>> out = bn(input)
"""

class BatchNorm1d(torch.nn.Module):
  def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
    super().__init__()
    
    self.in_features = in_features
    self.track_running_stats = track_running_stats
    self.affine = affine
    self.momentum = momentum
    
    if self.affine:
      self.gamma = torch.nn.Parameter(torch.ones(self.in_features, 1))
      self.beta = torch.nn.Parameter(torch.zeros(self.in_features, 1))
    
    if self.track_running_stats:
      # register_buffer registers a tensor as a buffer that will be saved as part of the model
      # but which does not require to be trained, differently from nn.Parameter
      self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
      self.register_buffer('running_std', torch.ones(self.in_features, 1))
  
  def forward(self, x):
    # transpose (N, C) to (C, N)
    x = x.transpose(0, 1).contiguous().view(x.shape[1], -1)
    
    # calculate batch mean
    mean = x.mean(dim=1).view(-1, 1)
    
    # calculate batch std
    std = x.std(dim=1).view(-1, 1)
    
    # during training keep running statistics (moving average of mean and std)
    if self.training and self.track_running_stats:
      # no computational graph is necessary to be built for this computation
      with torch.no_grad():
        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
        self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std
    
    # during inference time
    if not self.training and self.track_running_stats:
      mean = self.running_mean
      std = self.running_std
    
    # normalize the input activations
    x = (x - mean) / std
    
    # scale and shift the normalized activations
    if self.affine:
      x = x * self.gamma + self.beta
    
    return x.transpose(0, 1)

In [ ]:
"""
Applies Batch Normalization over a 2D or 3D input (4D tensor)

Shape:
  Input: (N, C, H, W)
  Output: (N, C, H, W)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm2d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm2d(4, affine=False)
  >>> input = torch.rand(10, 4, 5, 5)
  >>> out = bn(input)
"""

class BatchNorm2d(torch.nn.Module):
  def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
    super().__init__()
    
    self.in_features = in_features
    self.track_running_stats = track_running_stats
    self.affine = affine
    self.momentum = momentum
    
    if self.affine:
      self.gamma = torch.nn.Parameter(torch.ones(self.in_features, 1))
      self.beta = torch.nn.Parameter(torch.zeros(self.in_features, 1))
    
    if self.track_running_stats:
      # register_buffer registers a tensor as a buffer that will be saved as part of the model
      # but which does not require to be trained, differently from nn.Parameter
      self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
      self.register_buffer('running_std', torch.ones(self.in_features, 1))
  
  def forward(self, x):
    # transpose (N, C, H, W) to (C, N, H, W)
    x = x.transpose(0, 1)
    
    # store the shape
    c, bs, h, w = x.shape
    
    # collapse all dimensions except the 'channel' dimension
    x = x.contiguous().view(c, -1)
    
    # calculate batch mean
    mean = x.mean(dim=1).view(-1, 1)
    
    # calculate batch std
    std = x.std(dim=1).view(-1, 1)
    
    # keep running statistics (moving average of mean and std)
    if self.training and self.track_running_stats:
      with torch.no_grad():
        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
        self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std
    
    # during inference time
    if not self.training and self.track_running_stats:
      mean = self.running_mean
      std = self.running_std
    
    # normalize the input activations
    x = (x - mean) / std
    
    # scale and shift the normalized activations
    if self.affine:
      x = x * self.gamma + self.beta
    
    return x.view(c, bs, h, w).transpose(0, 1)


In [ ]:
def test_step_zero_shot_clip(net, data_loader, texts_z):
    samples = 0.0
    cumulative_accuracy = 0.0

    # set the network to evaluation mode
    net.eval()

    # disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # iterate over the test set
        for batch_idx, batch in enumerate(data_loader):
            inputs = batch['image']
            targets = batch['labels']
          # load data into GPU
            inputs = inputs.to(device)
            targets = targets.to(device)

          # forward pass
          # these two lines are different from the "traditional" ones
            images_z = model.encode_image(inputs).float()
   
            outputs = (100 * images_z @ texts_z.T).softmax(dim=-1)

            # fetch prediction and loss value
            samples += inputs.shape[0]
            _, predicted = outputs.max(1)

          # compute accuracy
            cumulative_accuracy += predicted.eq(targets).sum().item()
            print(cosine_similarity(images_z, texts_z))
            
    return cumulative_accuracy / samples * 100

In [ ]:
class CustomCLIP(torch.nn.Module):
  def __init__(self, num_classes: int = 10, bias=False):
    super().__init__()
    model, _ = clip.load("RN50")
    in_features = 1024
    out_features = 1024

    # take the visual encoder of CLIP
    # we also convert it to be 32 bit (by default CLIP is 16)
    self.encoder = model.visual.float()
    #self.classifier = torch.nn.Linear(1024, num_classes)
    
    # add a bottleneck
    self.classifier = torch.nn.Sequential(
      torch.nn.Linear(in_features, in_features // 2, bias=bias),
      torch.nn.BatchNorm1d(512),
      torch.nn.ReLU(inplace=True),
      torch.nn.Linear(in_features // 2, in_features // 2, bias=bias),
      torch.nn.BatchNorm1d(512),
      torch.nn.ReLU(inplace=True),
      torch.nn.Linear(in_features // 2, out_features, bias=bias),
      torch.nn.BatchNorm1d(1024)
    )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.encoder(x)
    x = self.classifier(x)

    return x

In [ ]:
def pad_sequence(batch):
    return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0)
def my_collate_fn(batch):
    #batch = defaultdict(filter(lambda x: x is not None, batch))
    #images = torch.stack([batch['image']])
    #labels = torch.tensor(batch['labels'])
    images, labels = [], []
    for key in batch:
        if key=='image':
            images += torch.stack([batch[key]])
        if key=='labels':
            labels += [torch.tensor(batch[key])]
    #images = images.permute(1,0,2,3)
    images = pad_sequence(images)
    labels = pad_sequence(labels)
    #return { 'image': torch.stack([x['image'] for x in batch]), 'labels': torch.tensor([x['labels'] for x in batch]) }
    print(batch)
    return {'image': images, 'labels': labels}

In [ ]:
def get_data(data_dir, batch_size=64, transform=True, target_transform=True, test_batch_size=256):

  
    if transform:
        # convert the PIL images to Tensors
        transform = preprocess
    else:
          # prepare data transformations and then combine them sequentially
        transform = T.Compose([torchvision.transforms.ToTensor()]) 
    if target_transform:   
        target_transform = T.Compose([
                                 lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)])
    else:
        target_transform = None

  # load data
    dataset = preProcess_datasets(data_dir, train=True)
    full_training_data = RefCOCOgDataset(dataset=dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)

    test_dataset = preProcess_datasets(data_dir, train=False)
    test_data = RefCOCOgDataset(dataset=test_dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)
    
    texts = [v for k,v in enumerate(test_dataset.values())]
    with torch.no_grad():
      texts_z = model.encode_text(text_tokens).float()
      texts_z /= texts_z.norm(dim=-1, keepdim=True)
      texts_z_views.append(texts_z)
    
    # evaluate the mean representation
    texts_z = torch.stack(texts_z_views).mean(dim=0)

    # renormalise
    texts_z /= texts_z.norm(dim=-1, keepdim=True)
  
    
    evens = list(range(0, len(test_dataset), 2))
    sample_dataset = torch.utils.data.Subset(test_dataset, evens)
    #images_z, texts_z = encode_data([v for k,v in enumerate(test_dataset.keys())], [v for k,v in enumerate(test_dataset.values())], data_dir=data_dir)

  # create train and validation splits
    num_samples = len(full_training_data)
    training_samples = int(num_samples * 0.5 + 1)
    validation_samples = num_samples - training_samples

    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

  # initialize dataloaders_collate
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=2, collate_fn=my_collate_fn)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)

    # pre-train model on zero-shot transfer learning
    test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z)


    # evaluate accuracy on zero-shot learning

    print("Test accuracy {:.2f}".format(test_accuracy))

    return train_loader, val_loader, test_loader
    #return full_training_data

In [ ]:
# tensorboard logging utilities
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [ ]:
# main funcition
def main(
      root='/content/dataset/refcocog/',
      data_dir='dataset/refcocog',
      batch_size=128,
      num_classes=10,
      learning_rate=0.01,
      weight_decay=0.000001,
      momentum=0.9,
      epochs=10,
    ): 
  writer = SummaryWriter(log_dir="runs/exp1")

  # train clip on zero-shot learning and instantiates dataloaders
  train_loader, val_loader, test_loader = get_data(data_dir=data_dir, batch_size=batch_size, transform=True, target_transform=True, test_batch_size=None)
  #dataset = get_data(data_dir=data_dir, batch_size=batch_size, transform=True, test_batch_size=None)

  # instantiate the network and move it to the chosen device (GPU)
  modified_model = CustomCLIP(num_classes=num_classes).to(device)

  # instantiate the optimizer
  optimizer = get_optimizer(modified_model, learning_rate, weight_decay, momentum)
  
  # define the cost function
  cost_function = get_cost_function()

  # evaluate accuracy of modified model on zero-shot learning
  #test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z).to(device)
  
  #print(cosine_similarity(images_z, texts_z))
  #print("Test accuracy {:.2f}".format(test_accuracy))

  # computes evaluation results before training
  print('Before training:')
  train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
  val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
  test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

  # log to TensorBoard
  log_values(writer, -1, train_loss, train_accuracy, "train")
  log_values(writer, -1, val_loss, val_accuracy, "validation")
  log_values(writer, -1, test_loss, test_accuracy, "test")

  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # for each epoch, train the network and then compute evaluation results
  for e in range(epochs):
    
    train_loss, train_accuracy = training_step(modified_model, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)

    # logs to TensorBoard
    log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

    # compute final evaluation results
    print('After training:')
    train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
    test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

    # log to TensorBoard
    log_values(writer, epochs, train_loss, train_accuracy, "train")
    log_values(writer, epochs, val_loss, val_accuracy, "validation")
    log_values(writer, epochs, test_loss, test_accuracy, "test")

    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')

  # closes the logger
  writer.close()


In [ ]:
!pip install pynvml
print(torch.cuda.list_gpu_processes())
torch.cuda.empty_cache()
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<32>"

GPU:0
process       6853 uses      980.000 MB GPU memory


In [ ]:
main()

In [ ]:
a = { 'b': [0,1], 'c': [1]}
print(list(a.items())[0][1])
print(v for k,v in enumerate((tuple(a.items())[0])))
d = defaultdict(list)
d['p'] = list(a.items())[1]
print(list(d.items())[0][0])
o = {'COCO_train2014_000000131074.jpg': ['a bed in the bedroom with other bed also , on its right side', 'a long bed at the side of wall']}
print([v for k,v in enumerate(o.keys())])

clip.tokenize([sent for desc in o.values() for sent in desc])
